In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.ml.linalg import Vectors,VectorUDT
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.tuning import CrossValidatorModel

In [2]:
spark = SparkSession.builder.appName("comment_predict").master("local[*]").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-avro_2.12 added as a dependency
org.elasticsearch#elasticsearch-spark-30_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-49107c41-72e9-439e-a7e9-23bc0735b505;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.0 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.

In [3]:
import subprocess

svm_model = CrossValidatorModel.load("hdfs://namenode:9000/workspace/output/model/svm_model")
cmd = ["hdfs","dfs","-ls","-R","/workspace/output/comments"]
result = subprocess.run(cmd,capture_output=True,text=True)
lines = result.stdout.strip().split("\n")
parquet_files = [line.split()[-1] for line in lines if line.endswith(".parquet")]
df = spark.read.parquet("hdfs://namenode:9000/workspace/output/comments/comment_gt128_lte256_part_1.parquet")
vector_col = [col for col in df.columns if col.startswith("v")]
assembler = VectorAssembler(inputCols=vector_col,outputCol="features")
to_vector_udf = udf(lambda arr : Vectors.dense(arr) , VectorUDT())
for parquet_file in parquet_files:
    df = spark.read.parquet(f"hdfs://namenode:9000{parquet_file}")
    df = assembler.transform(df)
    df = df.withColumn("features",to_vector_udf("features"))
    df_pred = svm_model.transform(df)
    save_path = parquet_file.split("/")[-1].split(".parquet")[0]
    df_pred.write.mode("overwrite").parquet(f"hdfs://namenode:9000/workspace/output/prediction/svm_pred/{save_path}")

In [5]:
spark.stop()